# This notebook provides a demonstration of pulling data from two different HAPI servers.

In [ ]:
from hapiclient import hapi
from hapiclient import hapitime2datetime
import json
import numpy as np
import datetime
import matplotlib
from matplotlib import pyplot as plt

%matplotlib inline

### Let numpy objects full print, even if they are long

In [ ]:
np.set_printoptions(threshold=np.inf)

### Define the data we want to get, including the HAPI server
See http://hapi-server.org/servers/ for a list of other HAPI servers and datasets.

In [ ]:
server     = 'https://cdaweb.gsfc.nasa.gov/hapi'
dataset    = 'OMNI2_H0_MRG1HR'
start      = '2014-12-20T00:00:00'
stop       = '2014-12-23T00:00:00'
parameters = 'DST1800'
opts       = {'logging': False}

### Get data from the remote server.
Print a message when done (may geet some HTTP errors since this server has no certificate)

In [ ]:
data1, meta1 = hapi(server, dataset, parameters, start, stop, **opts)
print("Loaded data: {} -> {}".format(dataset, parameters))

### Show the metadata fro this dataset.
This just prettyprints the JSON content.

In [ ]:
print(json.dumps(meta1,indent=4))

### Convert time to Python datetime values.
Data comes back as a numpy ndarray.

Each parameter in the returned data object is available as a named element based on the parameter name.

Here we are just getting shorter names for the named numeric elements.

In [ ]:
t1 = hapitime2datetime(data1["Time"])
d1 = data1["DST1800"]
print(d1)

### Let's make a simple plot!

In [ ]:
plt.plot(t1,d1)
plt.gcf().autofmt_xdate()
plt.show()

### Now get more data from a DIFFERENT HAPI server.

Use the same time range.

*NOTE* This takes about 10 to 15 seconds to load the data.


In [ ]:

server = "http://amda.irap.omp.eu/service/hapi"
dataset = "clust4-cis-prp"
parameters = "c4_h_dens,c4_h_v,c4_h_t"
data2, meta2 = hapi(server, dataset, parameters, start, stop, **opts)
print("Loaded data: {} -> {}".format(dataset, parameters))

print(json.dumps(meta2,indent=4))
print(data2[:30])


### Now plot this other data.

In [ ]:
t2 = hapitime2datetime(data2["Time"])
d2 = data2["c4_h_dens"]
plt.plot(t2,d2)
plt.gcf().autofmt_xdate()
plt.show()

### Notice all the FILL data.
Remove the fill value and replot.
Get the fill value right from the metadata.
Also get the parameter names for the axes.

In [ ]:
# parameters are ordered; we want the fill value for the first one
fill = float(meta2["parameters"][1]["fill"])
print("fill value is {}".format(fill))
idx = np.where(d2 != fill)
# get reduced time and data arrays with no fill values:
t2r = t2[idx]
d2r = d2[idx]

# now plot these with Dst above
fig = plt.figure(figsize=[20,8])
axes = fig.subplots(2)
plt.gcf().autofmt_xdate()
axes[1].set_xlabel(meta2["parameters"][0]["name"])
axes[1].set_ylabel(meta2["parameters"][1]["name"])
axes[1].plot(t2r,d2r) # h density
axes[0].plot(t1,d1)  # Dst
plt.show()



### Get ephemeris data from SSCWeb
Also clean it, just in case.

In [ ]:
server = "http://hapi-server.org/servers/SSCWeb/hapi"
dataset = "cluster4"
parameters = "Lon_GSM,Radius"
data3, meta3 = hapi(server, dataset, parameters, start, stop, **opts)
time_name = meta3["parameters"][0]["name"]
dataIdx = 2
data_name = meta3["parameters"][dataIdx]["name"]
t3 = hapitime2datetime(data3[time_name])
d3 = data3[data_name]

fill = float(meta3["parameters"][dataIdx]["fill"])
idx3 = np.where(d3 != fill)
# get reduced time and data arrays with no fill values:
t3r = t3[idx3]
d3r = d3[idx3]


### Now plot all three data sources (each from different HAPI server) on the same plot.

In [ ]:
plt.rcParams.update({'font.size': 15})

fig = plt.figure(figsize=[20,8])
axes = fig.subplots(3)
plt.gcf().autofmt_xdate()
axes[0].set_ylabel(meta1["parameters"][1]["name"])
axes[0].plot(t1,d1)  # Dst

axes[1].set_ylabel(meta2["parameters"][1]["name"])
axes[1].plot(t2r,d2r) # h density

axes[2].set_xlabel(meta3["parameters"][0]["name"])
axes[2].set_ylabel(meta3["parameters"][dataIdx]["name"] + " " + meta3["parameters"][dataIdx]["units"])
axes[2].plot(t3r,d3r) # radius
plt.show()

